In [ ]:
! pip install -r requirements.txt

## Implant Aligner
This Notebook presents the code for the model responsible to align Implants in PA Radiographs with respect to the vertical axis of the image

### Import

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import random
from skimage.transform import resize
import pandas as pd

### Dataset

The dataset contains all the PA radiographs as `X` and the angle of their implant with respect to the vertical axis as `Y`.

#### Data Directory Structure:

- `Dataset/`
  - `images/`
  - `labels.csv`

#### Sample from `labels.csv`:

| images       | labels        |
|--------------|---------------|
| image1.jpg   | -3.099632121  |
| image2.jpg   | 7.172982666   |
| image3.jpg   | 13.27064388   |
| ...          | ...           |


In [2]:
print(len(os.listdir("Dataset/images")))

2883


### Utility functions

In [2]:
from PIL import Image, ExifTags
import os

def normalize_orientation(image):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = image._getexif()
        if exif is not None:
            exif = dict(exif.items())
            orientation_value = exif.get(orientation, None)

            if orientation_value == 3:
                image = image.rotate(180, expand=True)
            elif orientation_value == 6:
                image = image.rotate(270, expand=True)
            elif orientation_value == 8:
                image = image.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # cases: image don't have getexif
        pass
    return image

def convert_grayscale_to_rgb(input_path, output_path):
    # Open the image
    image = Image.open(input_path)

    # Normalize the orientation
    image = normalize_orientation(image)

    # Convert to grayscale
    grayscale_image = image.convert("L")

    # Convert to RGB
    rgb_image = Image.merge("RGB", (grayscale_image, grayscale_image, grayscale_image))

    # Save the RGB image
    rgb_image.save(output_path)


def process_images_in_directory(src_directory, dest_directory):
    # Loop through all files in the source directory and its subdirectories
    for root, dirs, files in os.walk(src_directory):
        for file in files:
            # Check if the file is an image (you can adjust the condition based on your file types)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tif','.tiff')):
                src_path = os.path.join(root, file)
                relative_path = os.path.relpath(src_path, src_directory)
                dest_path = os.path.join(dest_directory, relative_path)

                # Ensure the subdirectory structure exists in the destination directory
                os.makedirs(os.path.dirname(dest_path), exist_ok=True)

                # Convert and save the image
                convert_grayscale_to_rgb(src_path, dest_path)



### Data Augmentation and preprocessing

In [5]:
# Define the path to the folder containing TIFF images
image_folder = 'Dataset/images'  # ____________________________ Image mode selection

# Define the path to the CSV file containing angle labels
labels_file = 'Dataset/labels.csv'

def leftRightShift(image):
    image_array = np.array(image)
    # Shifting Augmentation
    shift_amount = random.randint(10, 30)
    shifted_right = np.concatenate((image_array[:, -shift_amount:], image_array[:, :-shift_amount]), axis=1)


    # Left shift
    shift_amount = random.randint(10, 30)
    shifted_left = np.concatenate((image_array[:, shift_amount:], image_array[:, :shift_amount]), axis=1)

    return np.array(shifted_right) / 255.0, np.array(shifted_left) / 255.0


def load_and_preprocess_images(image_folder, labels_file):
    images = []
    angles = []

    # Load the CSV file into a DataFrame
    df = pd.read_csv(labels_file)

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        filename = row['images']
        angle = row['labels']
        image_path = os.path.join(image_folder, filename)

        # Load image and convert to grayscale if necessary
        image = Image.open(image_path)
        image = normalize_orientation(image)
        # print(np.shape(image))
        if image.mode != 'L':
            image = image.convert('L')

        # Resize image to a fixed size (e.g., 128x128)
        image = image.resize((128, 128))   # ___________ img size ____________________

        # Convert image to numpy array and normalize pixel values
        image = np.array(image) / 255.0

        # Append image and angle label to lists
        images.append(image)
        angles.append(float(angle))

    return np.array(images), np.array(angles)

def load_augment_and_preprocess_images(image_folder, labels_file):
    images = []
    angles = []

    # Load the CSV file into a DataFrame
    df = pd.read_csv(labels_file)

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        filename = row['images']
        angle = row['labels']
        image_path = os.path.join(image_folder, filename)

        # Load image and convert to grayscale if necessary
        image = Image.open(image_path)
        image = normalize_orientation(image)
        # If the image is not in grayscale, convert it
        if image.mode != 'L':
            image = image.convert('L')

        # Resize image to a fixed size (e.g., 128x128)
        image = image.resize((128, 128))
        # image = image.resize((256, 256))

        # Convert image to numpy array and normalize pixel values

        # Append image and angle label to lists
        images.append(np.array(image) / 255.0) # original image
        angles.append(float(angle))  # Assuming angles are represented as floating-point numbers
        # aug#2
        rand_angle = random.randint(0, 15) 
        rotated_image_pos = image.rotate(rand_angle)  
        images.append(np.array(rotated_image_pos) / 255.0)
        angles.append(float(angle) + rand_angle)
        # aug#3,4
        # Additional Left Right on rotated
        right, left = leftRightShift(rotated_image_pos)
        images.append(np.array(right) / 255.0)
        images.append(np.array(left) / 255.0)
        angles.append(float(angle) + rand_angle)
        angles.append(float(angle) + rand_angle)
        # aug# 5
        rand_angle = random.randint(-15, 0)
        rotated_image_pos = image.rotate(rand_angle)
        images.append(np.array(rotated_image_pos) / 255.0)
        angles.append(float(angle) + rand_angle)
        # aug# 6,7
        # Additional Left Right on rotated
        right, left = leftRightShift(rotated_image_pos)
        images.append(np.array(right) / 255.0)
        images.append(np.array(left) / 255.0)
        angles.append(float(angle) + rand_angle)
        angles.append(float(angle) + rand_angle)
        # aug#8
        rand_angle = random.randint(15, 30)
        rotated_image_pos = image.rotate(rand_angle)
        images.append(np.array(rotated_image_pos) / 255.0)
        angles.append(float(angle) + rand_angle)
        # aug# 9, 10
        # Additional Left Right on rotated
        right, left = leftRightShift(rotated_image_pos)
        images.append(np.array(right) / 255.0)
        images.append(np.array(left) / 255.0)
        angles.append(float(angle) + rand_angle)
        angles.append(float(angle) + rand_angle)

        # aug# 11
        rand_angle = random.randint(-30, -15)
        rotated_image_pos = image.rotate(rand_angle)
        images.append(np.array(rotated_image_pos) / 255.0)
        angles.append(float(angle) + rand_angle)
        # aug# 12, 13
        # Additional Left Right on rotated
        right, left = leftRightShift(rotated_image_pos)
        images.append(np.array(right) / 255.0)
        images.append(np.array(left) / 255.0)
        angles.append(float(angle) + rand_angle)
        angles.append(float(angle) + rand_angle)

        image_array = np.array(image)
        # Shifting Augmentation
        shift_amount = random.randint(10, 30)
        shifted_right = np.concatenate((image_array[:, -shift_amount:], image_array[:, :-shift_amount]), axis=1)
        # shifted_right = image[:, -shift_amount:]  # Example: Shift by 20 pixels
        images.append(np.array(shifted_right) / 255.0)
        angles.append(float(angle))  # Angle remains unchanged

        # aug# 14, 15
        # Left shift
        shift_amount = random.randint(10, 30)
        shifted_left = np.concatenate((image_array[:, shift_amount:], image_array[:, :shift_amount]), axis=1)
        shifted_left = np.concatenate((image_array[:, shift_amount:], np.zeros((image_array.shape[0], shift_amount))), axis=1)
        # shifted_left = image[:, shift_amount:]  # Example: Shift by 20 pixels
        images.append(np.array(shifted_left) / 255.0)
        angles.append(float(angle))  # Angle remains unchanged

        # Zooming Augmentation
        zoom_factor = random.uniform(0.8, 1.2)  # Random zoom factor between 0.8 and 1.2
        zoomed_image = resize(image_array, (image_array.shape[0], int(image_array.shape[1] * zoom_factor)), anti_aliasing=True)

        # If zoomed in, crop the image
        if zoom_factor > 1:
            zoomed_image = zoomed_image[:, :image_array.shape[1]]
        # If zoomed out, pad the image with black pixels
        elif zoom_factor < 1:
            pad_width = image_array.shape[1] - zoomed_image.shape[1]
            zoomed_image = np.pad(zoomed_image, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
        # aug# 16
        images.append(np.array(zoomed_image) / 255.0)
        angles.append(float(angle))  # Angle remains unchanged
        # aug# 17
        # Brightness adjustment
        brightness_factor = random.uniform(0.7, 1.3)  # Randomly select brightness factor between 0.8 to 1.2
        image_array_nor = image_array / 255.0
        brightened = np.clip(image_array_nor * brightness_factor, 0, 1)
        images.append(brightened)
        angles.append(float(angle))  # Angle remains unchanged}
# ________________________________________________________________
    return np.array(images), np.array(angles)


augment = True  # Whether to augment or not

if augment is True:
    images, angles = load_augment_and_preprocess_images(image_folder, labels_file)
else:
    images, angles = load_and_preprocess_images(image_folder, labels_file)


In [6]:
print(np.shape(images))
print(np.shape(angles))

(40307, 128, 128)
(40307,)


### Train, Val and Test split

In [7]:
# Split the dataset into training, validation, and test sets
train_images, test_images, train_angles, test_angles = train_test_split(images, angles, test_size=0.1, random_state=42)
train_images, val_images, train_angles, val_angles = train_test_split(train_images, train_angles, test_size=0.1, random_state=42)

In [10]:
print(np.shape(val_images))

(3628, 128, 128)


In [11]:
np.shape(train_images)

(32648, 128, 128)

In [12]:
# Define the CNN model
def create_regression_cnn(input_shape):
  model = models.Sequential([
      # Convolutional layers
      layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(128, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(256, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(512, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(512, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Flatten(),
      # Dense layers for regression
      layers.Dense(1024, activation='relu'),
      layers.Dense(512, activation='linear'),
      layers.Dense(256, activation='linear'),
      layers.Dense(1, activation = 'linear')  # Output layer for regression
      ])
  return model

# Define the CNN model
def create_complex_regression_cnn(input_shape):
  model = models.Sequential([
      # Convolutional layers
      layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding = 'same'),
      layers.Conv2D(64, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(128, (3, 3), activation='relu', padding = 'same'),
      layers.Conv2D(128, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(256, (3, 3), activation='relu'),
      # layers.Conv2D(256, (3, 3), activation='relu', padding = 'same'),
      layers.MaxPooling2D((2, 2)),

      # layers.Conv2D(512, (3, 3), activation='relu', padding = 'same'),
      layers.Conv2D(512, (3, 3), activation='relu'),

      layers.MaxPooling2D((2, 2)),
      # layers.Conv2D(512, (3, 3), activation='relu', padding = 'same'),
      layers.Conv2D(512, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Flatten(),
      # Dense layers for regression
      layers.Dense(1024, activation='relu'),
      layers.Dense(512, activation='linear'),
      layers.Dense(256, activation='linear'),
      layers.Dense(1)  # Output layer for regression
      ])
  return model

  # Instantiate the model
# input_shape = (256, 256, 1)  # Specify your image dimensions and channels
input_shape = (128, 128, 1)  # Specify your image dimensions and channels

complex_regresion = True  # Whether to create simple regression model or complex

if complex_regresion is True:
  model = create_complex_regression_cnn(input_shape)
else:
  model = create_regression_cnn(input_shape)

model.summary()


d:\262541-Faizan\Implant bone loss project\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 126, 126, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 63, 63, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 61, 61, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,848,705 (26.13 MB)

 Trainable params: 6,848,705 (26.13 MB)

 Non-trainable params: 0 (0.00 B)

### Calbacks and model training

In [ ]:

#Callback

callback = callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    mode='min',          # Minimize validation loss
    min_delta=0.001,     # Minimum change to qualify as an improvement
    patience=50,         # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore weights from the epoch with the best validation loss
)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
metrics = model.fit(train_images,
                    train_angles,
                    epochs=1000,
                    batch_size=128,
                    validation_data=(val_images, val_angles),
                    callbacks=[callback])

# batch_size = 64
# metrics = model.fit(generator,
#                     steps_per_epoch=len(images) // batch_size,
#                     epochs=100,
#                     callbacks=[callback],
#                     validation_data=validation_generator)

# Evaluate the model
loss, mae = model.evaluate(test_images, test_angles)
print("Test Mean Squared Error:", loss)
print("Test Mean Absolute Error:", mae)

# Make predictions
predictions = model.predict(test_images)

Epoch 1/1000
256/256 [==============================] - 68s 191ms/step - loss: 358.5827 - mae: 14.8392 - val_loss: 338.6535 - val_mae: 14.3142
Epoch 2/1000
256/256 [==============================] - 46s 180ms/step - loss: 275.5250 - mae: 12.7260 - val_loss: 222.5983 - val_mae: 11.4997
Epoch 3/1000
256/256 [==============================] - 46s 178ms/step - loss: 171.1587 - mae: 9.7398 - val_loss: 141.3869 - val_mae: 8.5833
Epoch 4/1000
256/256 [==============================] - 46s 179ms/step - loss: 110.4719 - mae: 7.4453 - val_loss: 119.2678 - val_mae: 7.9916
Epoch 5/1000
256/256 [==============================] - 46s 179ms/step - loss: 82.5531 - mae: 6.2996 - val_loss: 94.3711 - val_mae: 6.7416
Epoch 6/1000
256/256 [==============================] - 46s 179ms/step - loss: 59.2954 - mae: 5.3389 - val_loss: 76.2022 - val_mae: 5.7921
Epoch 7/1000
256/256 [==============================] - 46s 178ms/step - loss: 44.1362 - mae: 4.6544 - val_loss: 68.5120 - val_mae: 5.4033
Epoch 8/1000
25

### Validation

In [14]:
# Evaluate the model
loss, mae = model.evaluate(test_images, test_angles)
print("Test Mean Squared Error:", loss)
print("Test Mean Absolute Error:", mae)

# Make predictions
predictions = model.predict(test_images)

126/126 ━━━━━━━━━━━━━━━━━━━━ 11s 84ms/step - loss: 25.2238 - mae: 2.9695
Test Mean Squared Error: 25.507320404052734
Test Mean Absolute Error: 3.023515462875366


### Prediction on test images

In [ ]:
import pandas as pd

# Evaluate the model
loss, mae = model.evaluate(test_images, test_angles)
print("Test Mean Squared Error:", loss)
print("Test Mean Absolute Error:", mae)

# Make predictions
predictions = model.predict(test_images)

# Create a DataFrame to visualize the differences
results_df = pd.DataFrame({
    'Test Angles': test_angles,
    'Predicted Angles': predictions.flatten()
})

# Print the DataFrame
print(results_df)


8/8 [==============================] - 0s 11ms/step - loss: 63.0958 - mae: 3.6835
Test Mean Squared Error: 63.09576416015625
Test Mean Absolute Error: 3.683526039123535
8/8 [==============================] - 0s 10ms/step
     Test Angles  Predicted Angles
0      -7.145106        -12.049554
1      -1.005086         -3.567605
2      15.561510         19.468838
3      -0.505538         -0.915460
4      13.279365         13.359903
..           ...               ...
233   -11.967102        -20.426786
234   -15.655782        -15.408672
235     1.901236         -0.040355
236    -0.309704          1.083019
237     7.731785          5.420019

[238 rows x 2 columns]


In [ ]:
results_df

,Test Angles,Predicted Angles
0,-7.145106,-7.335417
1,-1.005086,-1.342367
2,15.561510,12.261539
3,-0.505538,-0.980649
4,13.279365,13.287255
...,...,...
233,-11.967102,-16.406130
234,-15.655782,-15.185789
235,1.901236,2.879450
236,-0.309704,-3.128719


from matplotlib import pyplot as plt
results_df['Test Angles'].plot(kind='hist', bins=20, title='Test Angles')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df['Predicted Angles'].plot(kind='hist', bins=20, title='Predicted Angles')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df.plot(kind='scatter', x='Test Angles', y='Predicted Angles', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df['Test Angles'].plot(kind='line', figsize=(8, 4), title='Test Angles')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
results_df['Predicted Angles'].plot(kind='line', figsize=(8, 4), title='Predicted Angles')
plt.gca().spines[['top', 'right']].set_visible(False)

## Rotation Function for directory

In order to load the pretrained model, run the following command

In [ ]:
model = models.load_model('/content/drive/MyDrive/DMFT_datasets/Implant_bone_loss/implant_aligner_2371Withaug_WOrgb263epoch_cpu.keras')

In [ ]:
# Prediction function

def process_image(image_path):
  image_arr = []
  # Load image and convert to grayscale if necessary
  image = Image.open(image_path)
  # If the image is not in grayscale, convert it
  if image.mode != 'L':
      image = image.convert('L')

  image = normalize_orientation(image)

  # rgb_image = Image.merge("RGB", (grayscale_image, grayscale_image, grayscale_image)) # __________converting to rgb
  # Resize image to a fixed size (e.g., 128x128)
  image = image.resize((128, 128))

  # Convert image to numpy array and normalize pixel values
  image = np.array(image) / 255.0

  # Append image and angle label to lists
  image_arr.append(image)
  return np.array(image_arr)



In [ ]:
# rotation function

def rotate_image(image_path, angle, output_path):
  image_arr = []
  # Load image and convert to grayscale if necessary
  image = Image.open(image_path)

  rot_img = image.rotate(angle, Image.NEAREST, expand = 1)
  rot_img.save(os.path.join(output_path, os.path.basename(image_path)))

In [ ]:
output_path = 'path/to/Directory_containing_PA_images'

input_path = 'path/to/aligned_output_images'
os.makedirs(output_path, exist_ok= True)

# for x in range(151,169,1):
for img in os.listdir(input_path):
  path = os.path.join(input_path,img)
  arr = process_image(path)
  # print(np.shape(arr))
  pred_arr = arr.reshape((1, 128, 128, 1))
  predictions = model.predict(pred_arr)
  rotate_image(path,predictions[0][0],output_path)
  print(os.path.basename(path))
  print(predictions[0])

1/1 [==============================] - 0s 19ms/step
Implant internet (684).jpg
[5.815711]
1/1 [==============================] - 0s 17ms/step
Implant internet (724).jpg
[-6.554394]
1/1 [==============================] - 0s 17ms/step
Implant internet (598).jpg
[-11.357008]
1/1 [==============================] - 0s 18ms/step
Implant internet (99).jpg
[2.225787]
1/1 [==============================] - 0s 19ms/step
Implant internet (170).jpg
[5.770026]
1/1 [==============================] - 0s 18ms/step
Implant internet (389).jpg
[-1.0280272]
1/1 [==============================] - 0s 17ms/step
MBL Implant (53).tif
[-1.5198296]
1/1 [==============================] - 0s 17ms/step
MBL Implant (230).tif
[-3.6597884]
1/1 [==============================] - 0s 21ms/step
MBL Implant (152).tif
[3.6348686]
1/1 [==============================] - 0s 18ms/step
MBL Implant (189).tif
[1.4641862]
1/1 [==============================] - 0s 18ms/step
gen_implant (16).png
[-11.068403]
1/1 [====================